In [109]:
from scipy.stats import unitary_group
import bqskit
import cirq
from regulus.utils.functions import infidelity
from regulus.utils.arch import gene_random_circuit
import regulus
import numpy as np
from math import pi
from bqskit.ir.gates import *
from bqskit.compiler import Compiler
from regulus.ext.bqskit import bqskit_to_qasm
from regulus import gates
from tqdm import tqdm
from collections import Counter

In [77]:
def bqskit_to_cirq(circ: bqskit.Circuit) -> cirq.Circuit:
    circ = regulus.Circuit.from_qasm(bqskit_to_qasm(circ))
    return circ.to_cirq()

In [78]:
size = 1000
x = np.random.uniform(0, 0.5, size=size)
y = np.random.uniform(0, 0.5, size=size)
z = np.random.uniform(-0.5, 0.5, size=size)
idx = (x >= y) & (y >= np.abs(z))
x = x[idx]
y = y[idx]
z = z[idx]

In [117]:
target_gate_set = {SqrtISwapGate(), U3Gate()}
model = bqskit.MachineModel(2, gate_set=target_gate_set)
num_2q_gates_bqskit = []
num_2q_gates_cirq = []
qubits = cirq.LineQubit.range(2)
compiler = Compiler()
for coord in tqdm(zip(x, y, z), total=len(x)):
    
    # u = unitary_group.rvs(4)

    u = gates.Can(*np.array(coord) * pi).data

    circ = bqskit.compile(u, model=model, optimization_level=2, compiler=compiler)
    num_2q_gates_bqskit.append(circ.num_operations - circ.gate_counts[U3Gate()])

    num_2q_gates_cirq.append(np.count_nonzero(np.array([cirq.num_qubits(op) for op in (cirq.Circuit(cirq.two_qubit_matrix_to_sqrt_iswap_operations(*qubits, u)).all_operations())]) == 2))


100%|██████████| 160/160 [00:42<00:00,  3.81it/s]


In [116]:
np.array(num_2q_gates_bqskit) - np.array(num_2q_gates_cirq)

array([1, 2, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 2, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1])

In [118]:
np.array(num_2q_gates_bqskit) - np.array(num_2q_gates_cirq)


array([0, 2, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 2, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 2, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1])

In [126]:
idx = 6
x[idx], y[idx], z[idx]

(np.float64(0.07144794275360161),
 np.float64(0.06033400752153595),
 np.float64(0.038138377556605985))

In [125]:
num_2q_gates_cirq[:10]

[2, 2, 3, 2, 2, 2, 3, 3, 2, 3]

In [121]:
Counter(num_2q_gates_bqskit)

Counter({3: 125, 2: 24, 4: 11})

In [ ]:
Counter(num_2q_gates_bqskit)

Counter({2: 81, 3: 79})

In [88]:
qubits

[cirq.LineQubit(0), cirq.LineQubit(1)]

In [ ]:
Counter(num_2q_gates)

Counter({3: 112, 2: 38, 4: 10})

In [ ]:
cirq.two_qubit_matrix_to_sqrt_iswap_operations()

In [55]:
print('#2Q: {}'.format(circ.num_operations - circ.gate_counts[U3Gate()]))
print('gate_stats: {}, depth: {}'.format(circ.gate_counts, circ.depth))
print('infidelity: {}'.format(infidelity(circ.get_unitary(), u)))

#2Q: 2
gate_stats: {U3Gate: 6, SqrtISwapGate: 2}, depth: 5
infidelity: 1.1102230246251565e-16


In [35]:
# bqskit_to_cirq(circ)

In [ ]:
# circ_opt = regulus.Circuit.from_bqskit(circ_bqs_opt)
# print('num_gates: {}, num_2q_gates: {}, depth: {}'.format(circ_opt.num_gates,
#       circ_opt.num_nonlocal_gates, circ_opt.depth))
# print('infidelity: {}'.format(infidelity(circ_opt.unitary(), target)))

print(bqskit_to_qasm(circ_bqs_opt))



print('gate_stats: {}, depth: {}'.format(circ_bqs_opt.gate_counts, circ_bqs_opt.depth))
print('infidelity: {}'.format(infidelity(circ_bqs_opt.get_unitary(), target)))
# circ_bqs_opt.save('circ_bqs_opt.qasm')

for g in circ_bqs_opt.operations():
    if g.num_qudits == 2:
        print(g)

# print(circ_opt.to_qiskit().draw(fold=500))